# Dog vs. Cat Detection

## Import libraries

In [ ]:
#Pre do something

## Import Data
the next step is to import the data


In [ ]:
TRAIN_DIR = '../input/train/' #training directory
TEST_DIR = '../input/test/' #testing directory



Pre-define a function that will take the labels from the image name (thanks to other kernels on Kaggle).

In [ ]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1,0] #[cat, not dog]
    elif word_label == 'dog': return [0,1] #[not cat, dog]

## Retrieve training data


In [ ]:
from tqdm import tqdm      # percentage progress bar
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm 
IMG_SIZE=50 #Define short image size

def create_train_data():
    X = [] #training images
    Y = [] #training labels
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path=os.path.join(TRAIN_DIR, img)
        im=cv2.imread(path)
        sh=im.shape
        if sh[0] < sh[1]:  # Find the shortest dimension
            im1=im[0:sh[0],0:sh[0]]
            im2=im[0:sh[0],(sh[1]-sh[0]):sh[1]]
            #rint(""
        else:
            im1=im[0:sh[1],0:sh[0]]
            im2=im[(sh[0]-sh[1]):sh[0],0:sh[1]]
        #Resize images to make them smaller
        im1 = cv2.resize(im1, (IMG_SIZE,IMG_SIZE))
        im2 = cv2.resize(im2, (IMG_SIZE,IMG_SIZE))
        #Append the two images together
        imn=np.append(im1,im2,axis=0)
        X.append(np.array(imn))
        Y.append(np.array(label))
    return(X,Y)
    


Run the new function to create the training data.

In [ ]:
X,Y = create_train_data()


Try some of the images to make sure what they look like:

In [ ]:
import matplotlib.pyplot as plt
import random
index=random.randint(0,25000-1)
plt.imshow(X[index])
Y[index]
X[index].shape

X, Y = shuffle(X, Y)
Xtrain=X[0:20000-1]
Ytrain=Y[0:20000-1]
Xtest=X[(20000-1):25000-1]
Ytest=Y[(20000-1):25000-1]


## Define the Neural Network
This next section will define the neural network for the detector

In [ ]:
import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

#Pre-process the images
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)


# Convolutional network building
network = input_data(shape=[None, 100, 50, 3], #Define input layer
                     data_preprocessing=img_prep, #set data_preprocessing
                     data_augmentation=img_aug)

network = conv_2d(network, 32,3, activation='relu') #convolve data once
network = max_pool_2d(network,2) #down sample (reduce data)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network,2) #down sample (reduce data)
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 2, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

Run the training iterations repeatedly

In [ ]:
#Define the model
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='dog-cat.tfl.ckpt')
model.fit(Xtrain, Ytrain, n_epoch=3, 
          shuffle=True, 
          validation_set=(Xtest, Ytest),
          show_metric=True, 
          batch_size=96, 
          run_id='catvdog')

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, 2*IMG_SIZE, IMG_SIZE, 3], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


model.fit(Xtrain, Ytrain, 
          n_epoch=2, 
          validation_set=(Xtest, Ytest),
          snapshot_step=50000, 
          show_metric=True, 
          run_id='catvdog')